<a href="https://colab.research.google.com/github/chengeke/MITCreditLoanDefaults/blob/master/credit_approval_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Saving german_credit_data.csv to german_credit_data.csv
user uploaded file"german_credit_data.csv"with length 47 400 bytes
   Unnamed: 0  Age     Sex  ...  Credit amount  Duration              Purpose
0           0   67    male  ...           1169         6             radio/TV
1           1   22  female  ...           5951        48             radio/TV
2           2   49    male  ...           2096        12            education
3           3   45    male  ...           7882        42  furniture/equipment
4           4   53    male  ...           4870        24                  car

[5 rows x 10 columns]


TypeError: ignored

In [ ]:
from google.colab import files
uploaded=files.upload()
for fn in uploaded.keys():
  print('user uploaded file"credit_approval.csv"with length 28 747 bytes'.format(name=fn,length=len(uploaded[fn])))
 
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
import keras.utils
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
np.random.seed(3)
 
df=pd.read_csv('credit_approval.csv')
print(df.head())
print(df.shape)
 
%reload_ext tensorboard
%tensorboard --logdir logs/fit
X=df.drop('Target',axis=1)
Y=df.Target
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=5)
model = Sequential()
model.add(Dense(10, input_dim=14,  activation='relu'))

sgd=SGD(learning_rate=0.001,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['binary_accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


results=model.fit(x_train, y_train, validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback], epochs=20, batch_size=100)
epochs=20
epochs_arr = np.arange(1,epochs + 1, 1)
data_results =results.history
line1 = plt.plot(epochs_arr, data_results['loss'], 'r-', label='training loss')
line2 = plt.plot(epochs_arr, data_results['val_loss'], 'b-', label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model loss')
plt.legend()
plt.show()

line1 = plt.plot(epochs_arr,data_results['binary_accuracy'], 'r-', label='training accuracy')
line2 = plt.plot(epochs_arr,data_results['val_binary_accuracy'], 'b-', label='validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model accuracy')
plt.legend()
plt.show()

accuracy=model.evaluate(x_test,y_test)
print("accuracy=",(model.metrics_names[1],accuracy[1]*100))

X=df.drop('Target',axis=1)
Y=df.Target
X, Y = make_classification(n_samples=690, n_classes=2, random_state=1)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=5)

ns_probs = [0 for _ in range(len(y_test))]
model = LogisticRegression(solver='lbfgs')
model.fit(x_train, y_train)

lr_probs = model.predict_proba(x_test)
lr_probs = lr_probs[:, 1]
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
plt.title('ROC Plot for credit_approval ')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()